In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

#read the datasets
# train_df = pd.read_csv("./train.csv")
# val_df = pd.read_csv("./val.csv")
# test_df = pd.read_csv("./test.csv")
df = pd.read_csv("./df2.csv")
df = df.rename(columns={"Unnamed: 0": "label_id"})
text = df.text.values
labels = df.label_id.values
# train_df=train_df.rename(columns={"Unnamed: 0": "label_id"})
# val_df=val_df.rename(columns={"Unnamed: 0": "label_id"})
# test_df=test_df.rename(columns={"Unnamed: 0": "label_id"})
# train_text = train_df.text.values
# train_labels = train_df.label_id.values
# val_text = val_df.text.values
# val_labels = val_df.label_id.values
# test_text = test_df.text.values
# test_labels = test_df.label_id.values
# train_labels = train_labels.astype(float)
'''for idx in range(len(train_labels)):
    train_labels[idx] = train_labels[idx].astype('float32')
print(type(train_labels))
print(type(train_labels[0]))'''

#tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

#helper function to check tokenization of sentences
def print_rand_sentence():
  idx = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[idx]), tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[idx]))]).T
  print(tabulate(table, headers = ['Tokens', 'Token IDs'], tablefmt = 'fancy_grid'))

# print_rand_sentence()

#tokenize the corpus
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(input_text,add_special_tokens = True,  max_length = 64,padding = 'max_length',
                        return_attention_mask = True,return_tensors = 'pt',truncation=True)

for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)
# print(token_id[196])

# helper function to check encoding of sentences
def print_rand_sentence_encoding():
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]
  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, headers = ['Tokens', 'Token IDs', 'Attention Mask'],tablefmt = 'fancy_grid'))

# print_rand_sentence_encoding()

#create dataset
ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16
train_i, val_i = train_test_split(np.arange(len(labels)),test_size = ratio,shuffle = True)
# Train and validation sets
# train = TensorDataset(token_id[train_i], attention_masks[train_i], labels[train_i])
train = TensorDataset(token_id[:], attention_masks[:], labels[:])
val = TensorDataset(token_id[val_i], attention_masks[val_i], labels[val_i])
# Prepare DataLoader
train_dataloader = DataLoader(train,sampler = RandomSampler(train),batch_size = batch_size)
validation_dataloader = DataLoader(val,sampler = SequentialSampler(val),batch_size = batch_size)

#Main Training
#Define eval metrics
def b_tp(preds, labels):
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])
def b_fp(preds, labels):
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])
def b_tn(preds, labels):
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])
def b_fn(preds, labels):
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])
def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
            'bert-base-uncased',num_labels = len(labels),output_attentions = False,output_hidden_states = False,)
optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5,eps = 1e-08)


#Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# print(device)
epochs = 16
for _ in trange(epochs, desc = 'Epoch'):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask, labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    model.eval() 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')



In [ ]:
# !pip install transformers

In [ ]:
torch.cuda.is_available()

True

In [3]:
df = pd.read_csv("./df2.csv")
df = df.rename(columns={"Unnamed: 0": "label_id"})

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [5]:
text = df.text.values
labels = df.label_id.values

In [6]:
#tokenize the corpus
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(input_text,add_special_tokens = True,  max_length = 64,padding = 'max_length',
                        return_attention_mask = True,return_tensors = 'pt',truncation=True)

for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)
print(token_id[196])

tensor([  101,  1996,  8030,  2008,  4264,  8327,  1999,  3433,  2000,  1996,
        26835, 28775,  3723,  1997,  8418, 20738,  2863,  2137,  2819,  2024,
         2714,  2000,  2216,  1997,  2060, 22262, 14925, 14399,  5400, 28032,
         2594, 11265, 18900, 19847,  1997,  6147,  1012,  2009,  2003,  2691,
         2000,  2156,  3532,  3930,  1998,  2030, 15412,  2075,  1997,  1996,
         3269,  1010,  3756,  2075,  2030, 19863,  3436,  1997,  1996, 19624,
         1010,  1998,  4359,   102])


In [10]:
#create dataset
ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16
train_i, val_i = train_test_split(np.arange(len(labels)),test_size = ratio,shuffle = True)
# Train and validation sets
# train = TensorDataset(token_id[train_i], attention_masks[train_i], labels[train_i])
train = TensorDataset(token_id[:], attention_masks[:], labels[:])
val = TensorDataset(token_id[val_i], attention_masks[val_i], labels[val_i])
# Prepare DataLoader
train_dataloader = DataLoader(train,sampler = RandomSampler(train),batch_size = batch_size)
validation_dataloader = DataLoader(val,sampler = SequentialSampler(val),batch_size = batch_size)

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',num_labels=len(labels))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)

NameError: ignored